In [ ]:
import numpy as np
import os
from os import path


import torch
from torch import nn, optim
from torch.utils.data import DataLoader
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
import copy


In [ ]:
project_root = '/home/project/gas_anormaly_detection/restaurant/1training/'

In [ ]:
data_root = path.join(project_root, 'Dataset/')

save_weight_root = path.join(project_root, '3autoencoder/pretrained_weights/')
if not path.exists(save_weight_root):
    os.mkdir(save_weight_root)
    print("make a new dir.")
    

In [ ]:
#load training set

In [ ]:
train_samples = np.load(path.join(project_root, '2pattern_extraction/save_data/train_sample_inter.npy'))

# max normalization
for i in range(train_samples.shape[0]):
    train_sample_temp = train_samples[i]
    max_value = max(train_sample_temp)
    train_sample_temp = train_sample_temp/max_value
    train_samples[i] = train_sample_temp

## AE

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:

class cnn_Encoder(nn.Module):

    def __init__(self, seq_len, n_features, embedding_dim=64):
        super(cnn_Encoder, self).__init__()

        self.seq_len, self.n_features = seq_len, n_features
        self.embedding_dim= embedding_dim
        self.hidden_dim = 32

        self.cnn1 = nn.Conv1d(
          in_channels= self.n_features,
          out_channels=self.hidden_dim,
          kernel_size=4,
          padding=0,
          dilation=2,
          stride=2
        )

        self.cnn2 = nn.Conv1d(
          in_channels=self.hidden_dim,
          out_channels=self.hidden_dim,
          kernel_size=4,
          padding=0,
          dilation=2,
          stride=2
        )
        
        self.cnn3 = nn.Conv1d(
          in_channels=self.hidden_dim,
          out_channels=self.embedding_dim,
          kernel_size=8,
          padding=0,
          dilation=2,
          stride=3
        )
        self.fc = nn.Linear(self.embedding_dim*8, self.embedding_dim)

    def forward(self, x):
        
        x = x.transpose(1,2)
        
        x = self.cnn1(x)
        x = F.relu(x) 
        
        x = self.cnn2(x)
        x = F.relu(x)
        
        x = self.cnn3(x)
        x = F.relu(x)
       
        x = x.reshape(x.size(0),-1)
        x = self.fc(x)
        x = x.reshape(x.size(0),self.embedding_dim,1)

        return x

In [ ]:

class cnn_Decoder(nn.Module):

    def __init__(self, seq_len, input_dim=64, n_features=1):
        super(cnn_Decoder, self).__init__()
        
        self.seq_len, self.n_features = seq_len, n_features
        self.input_dim= input_dim


        self.cnn1 = nn.ConvTranspose1d(
          in_channels= self.input_dim,
          out_channels=64,
          kernel_size=4,
          dilation=2,
          stride=2)
        
        self.cnn2 = nn.ConvTranspose1d(
          in_channels= self.input_dim,
          out_channels=32,
          kernel_size=8,
          dilation=2,
          stride=2)
        
        self.cnn3 = nn.ConvTranspose1d(
          in_channels= 32,
          out_channels=16,
          kernel_size=8,
          dilation=3,
          stride=2)
        
        self.cnn4 = nn.ConvTranspose1d(
          in_channels= 16,
          out_channels=1,
          kernel_size=8,
          dilation=3,
          stride=2)

    def forward(self, x):
        x = self.cnn1(x)
        x = F.relu(x) 
        
        x = self.cnn2(x)
        x = F.relu(x) 
        
        x = self.cnn3(x)
        x = F.relu(x) 
        
        x = self.cnn4(x)
        
        x = x.transpose(1,2)
        return x

In [ ]:

class CNNAutoencoder(nn.Module):

    def __init__(self, seq_len, n_features, embedding_dim=64):
        super(CNNAutoencoder, self).__init__()

        self.encoder = cnn_Encoder(seq_len, n_features, embedding_dim).to(device)
        self.decoder = cnn_Decoder(seq_len, embedding_dim, n_features).to(device)

    def forward(self, x):
        x_embedding = self.encoder(x)
        x = self.decoder(x_embedding)

        return x_embedding, x

In [ ]:

def train_model(model, trainloader, valloader, n_epochs):
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
    
#     criterion = nn.L1Loss(reduction='mean').to(device)
    criterion = nn.CosineEmbeddingLoss(reduction='mean').to(device)
    
    history = dict(train=[], val=[])

    best_model_wts = copy.deepcopy(model.state_dict())
    best_loss = 10000000.0

    for epoch in range(1, n_epochs + 1):
        model = model.train()

        train_losses = []
        for index, data in enumerate(trainloader):
            optimizer.zero_grad()
            
            seq_true = data.float().to(device)
            __, seq_pred = model(seq_true)
            
            y = torch.ones([1]).to(device)
            loss = criterion(seq_pred, seq_true,y)

            loss.backward()
            optimizer.step()

            train_losses.append(loss.item())

        val_losses = []
        model = model.eval()
        with torch.no_grad():
            for index, data in enumerate(valloader):

                seq_true = data.float().to(device)
                __, seq_pred = model(seq_true)

                y = torch.ones([1]).to(device)
                loss = criterion(seq_pred, seq_true,y)
                
                val_losses.append(loss.item())
                

        train_loss = np.mean(train_losses)
        val_loss = np.mean(val_losses)


        history['train'].append(train_loss)
        history['val'].append(val_loss)

        if val_loss < best_loss:
            best_loss = val_loss
            best_model_wts = copy.deepcopy(model.state_dict())

        print(f'Epoch {epoch}: train loss {train_loss} val loss {val_loss}')

        file = open("./losses/cnnAE_restaurant.txt","a")
        file.write("Epoch = {}".format(epoch))
        file.write("\n")

        file.write("train_loss = {:.3f}  ".format(train_loss))
        file.write("\n")

        file.write("val_loss = {:.3f}   ".format(val_loss))
        file.write("\n")


        
    model.load_state_dict(best_model_wts)
    return model.eval(), history

In [ ]:
RANDOM_SEED=0
train_dataset, val_dataset = train_test_split(
  train_samples,
  test_size=0.1,
  random_state=RANDOM_SEED
)


train_dataset = np.array(train_dataset)
val_dataset = np.array(val_dataset)

train_dataset = train_dataset.reshape(train_dataset.shape[0],train_dataset.shape[1],1)
val_dataset = val_dataset.reshape(val_dataset.shape[0],val_dataset.shape[1],1)

print(train_dataset.shape)
print(val_dataset.shape)


In [ ]:
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4)
valloader = torch.utils.data.DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=4)


In [ ]:
seq_len=168
n_features=1
model = CNNAutoencoder(seq_len, n_features, hidden_size)

model = model.to(device)

In [ ]:
model, history = train_model(
  model, 
  trainloader, 
  valloader,
  n_epochs=200
)

In [ ]:
#save weights
model_state = {
            'net_state_dict': model.state_dict()
        }
       
save_point = save_weight_root+ '/cnnAE_restaurant'
torch.save(model_state, save_point)